In [1]:

from makiflow.layers import *
from makiflow.models.segmentation.segmentator import Segmentator
from makiflow.tf_scripts import set_main_gpu
set_main_gpu(1)
import tensorflow as tf
import numpy as np
import glob
import cv2
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
import random
from makiflow.models.classificator import Classificator

from sklearn.utils import shuffle
from scipy.ndimage import gaussian_filter

In [2]:
import copy
def merging_masks(masks, index_of_main_mask, priority):
    """
    We choose the base mask which have index `index_of_main_mask` on which is put other classes
    according to `priority` of other classes

    Parameters
    ----------
    masks : list or numpy.array
        List or numpy array of masks.
    index_of_main_mask : int
        Index of base mask.
    priority : list
        List of priority of classes. From highest priority to least priority.
        Example: [9, 10, 3, 7], where 9 class have highest priority and will be put on others classes,
        on the other hand the 7th class have the lowest priority and will be put on the others the latest.
    Returns
    ---------
    main_mask : numpy.array
        The base mask after merging other classes.
    """

    # For easy access
    main_mask = masks[index_of_main_mask]

    # Thanks to the `boolean_mask` higher priority class will not be erased
    boolean_mask = np.ones(main_mask.shape).astype(np.bool)

    for i in range(len(masks)):
        if i == index_of_main_mask:
            continue

        for prior in priority:
            main_mask[masks[i] == prior * boolean_mask] = prior
            boolean_mask[masks[i] == prior] = False

    return main_mask


def mutate_masks(masks, mapping):
    """
    Remaps classes on the given `masks` according to the `mapping`.

    Parameters
    ----------
    masks : list or numpy.array
        List or numpy array of masks.
    mapping : list
        List of tuples: [(source_class_number, new_class_number)],
        where `source_class_number` will be changed to `new_class_number` in the `masks`.

    Returns
    ---------
    new_masks : the same type as `masks`
        New masks with changed class numbers.
    """
    if type(mapping) is not list or (len(mapping) != 0 and type(mapping[0]) is not tuple):
        raise TypeError('mapping should be list of typles')

    new_masks = copy.deepcopy(masks)

    for i in range(len(new_masks)):
        for elem in mapping:
            old_value = elem[0]
            new_value = elem[1]
            new_masks[i][masks[i] == old_value] = new_value

    return  new_masks

In [3]:
from makiflow.save_recover import Builder

In [4]:
import tensorflow as tf

In [6]:
def load_test_data(path):
    Xtrain, Ytrain = [], []
    masks = glob.glob(path + '/masks/*.bmp')
    masks.sort()
    for mask_name in tqdm(masks):
        img = cv2.imread(mask_name.replace('masks', 'images'))
        img = cv2.resize(img, (1024, 1024), interpolation=cv2.INTER_CUBIC)
        mask = cv2.imread(mask_name)
        mask = mask[:,:,0]
        mask = cv2.resize(mask, (1024, 1024), interpolation=cv2.INTER_NEAREST)
        Xtrain.append(img)
        Ytrain.append(mask)
        if np.max(mask) >= 10:
            print(np.max(mask), f' in image {mask_name} ')
    return Xtrain, Ytrain

In [7]:
Xtest, Ytest = load_test_data(path='/mnt/data/med_data/balanced_batches/batch_3/test_set')

100%|██████████| 16/16 [00:00<00:00, 116.37it/s]


In [8]:
Xtest = [i / 255 for i in Xtest]

In [9]:
model = Builder.segmentator_from_json('other_classes/Xception_1024_testUnet_standart_V6.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

model.load_weights('other_classes/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=8/epoch_15/weights.ckpt')

answer_all = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

answer_all = np.argmax(answer_all,axis=3)

Model is restored!
INFO:tensorflow:Restoring parameters from other_classes/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=8/epoch_15/weights.ckpt
Weights are loaded.


In [10]:
answer_all = mutate_masks(answer_all,mapping=[(4,5),(5,6),(6,8),(7,9)])

In [11]:
model = Builder.segmentator_from_json('../exp_1/4_class_only/Xception_1024_testUnet_standart_V6.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

model.load_weights('../exp_1/4_class_only/exp_cv3/x-65/MakiSegmentator_gamma=6_opt_name=adam1_bsz=6/epoch_7/weights.ckpt')

answer_4 = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

Model is restored!
INFO:tensorflow:Restoring parameters from ../exp_1/4_class_only/exp_cv3/x-65/MakiSegmentator_gamma=6_opt_name=adam1_bsz=6/epoch_7/weights.ckpt
Weights are loaded.


In [12]:
answer_4 = np.argmax(answer_4, axis=3) * 4

In [13]:
model = Builder.segmentator_from_json('7_class_only/Xception_1024_testUnet_standart_V6.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

model.load_weights('7_class_only/exp_cv5_l2_v13/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=2/epoch_1/weights.ckpt')

answer_7 = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

answer_7 = np.argmax(answer_7,axis=3) * 7

Model is restored!
INFO:tensorflow:Restoring parameters from 7_class_only/exp_cv5_l2_v13/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=2/epoch_1/weights.ckpt
Weights are loaded.


In [35]:
answer_merged = []
for i in range(len(Xtest)):
    answer_merged.append(merging_masks([answer_all[i],answer_4[i],answer_7[i]],0,priority=[4,7]))

In [36]:
len(answer_merged)

16

In [37]:
from makiflow.metrics import categorical_dice_coeff, confusion_mat
from makiflow.metrics.utils import one_hot

In [17]:
name_classes = [ str(i) for i in range(10)]

In [38]:
answer_merged = np.asarray(answer_merged)

In [39]:
answer_merged = answer_merged.reshape(-1)
answer_merged = one_hot(answer_merged, depth=10)
answer_merged = answer_merged.reshape(16, -1, 10)

In [24]:
Ytest = np.asarray(Ytest)

In [40]:
EPSILON = 1e-9

def categorical_dice_coeff(P, L, use_argmax=False, ind_norm=True):
    """
    Calculates V-Dice for give predictions and labels.
    WARNING! THIS IMPLIES SEGMENTATION CONTEXT.
    Parameters
    ----------
    P : np.ndarray
        Predictions of a segmentator. Array of shape [batch_sz, W, H, num_classes].
    L : np.ndarray
        Labels for the segmentator. Array of shape [batch_sz, W, H]
    use_argmax : bool
        Converts the segmentator's predictions to one-hot format.
        Example: [0.4, 0.1, 0.5] -> [0., 0., 1.]
    ind_norm : bool
        Normalize each dice separately. Useful in case some classes don't appear
        on some images.
    """
    batch_sz = len(P)
    print(batch_sz)
    L = np.asarray(L)
    P = np.asarray(P)
    num_classes = P.shape[-1]
    if use_argmax:
        P = P.argmax(axis=3)
        P = P.reshape(-1)
        P = one_hot(P, depth=num_classes)
    P = P.reshape(batch_sz, -1, num_classes)
    L = L.reshape(batch_sz, -1)

    class_dices = np.zeros(num_classes)
    class_counts = np.zeros(num_classes) + EPSILON  # Smoothing to avoid division by zero
    for i in range(batch_sz):
        sample_actual = L[i]
        sample_pred = P[i]
        for j in range(num_classes):
            sub_actual = (sample_actual[:] == j).astype(np.int32)
            sub_confs = sample_pred[:, j]
            if np.sum(sub_actual) == 0 and np.sum(sub_confs) == 0:
                continue
            class_dices[j] += binary_dice(sub_confs, sub_actual)
            class_counts[j] += 1

    v_dice, dices = class_dices.mean() / batch_sz, class_dices / batch_sz
    if ind_norm:
        v_dice, dices = (class_dices / class_counts).mean(), class_dices / class_counts
    return v_dice, dices

def binary_dice(predicted, actual):
    num = np.sum(predicted * actual)
    den = np.sum(predicted * predicted) + np.sum(actual)
    return (2 * num + EPSILON) / (den + EPSILON)

In [49]:
# COMPUTE DICE AND CREATE CONFUSION MATRIX
v_dice_val, dices = categorical_dice_coeff(answer_merged, Ytest, use_argmax=False)

print('V-Dice:', v_dice_val)
for i, class_name in enumerate(name_classes):
    print(f'{class_name}:', dices[i])

# Compute and save matrix
conf_mat_path = f'mat_epoch.png'
print('Computing confusion matrix...')
confusion_mat(
    answer_merged, Ytest, use_argmax_p=True, to_flatten=True,
    save_path=conf_mat_path, dpi=175
)

16
V-Dice: 0.4689225858189707
0: 0.9560223898641762
1: 0.8452420646770852
2: 0.6679100722046011
3: 0.7287975416341761
4: 5.416421907765949e-13
5: 0.5555328724166723
6: 0.16403218445409734
7: 0.04081695362405828
8: 0.10132218148265787
9: 0.6295495978316412
Computing confusion matrix...


/home/rustam/anaconda3/lib/python3.6/site-packages/makiflow/metrics/metrics.py:170: RuntimeWarning: invalid value encountered in true_divide
  temp_mat = mat / mat.sum(axis=ax)


[array([[0.95, 0.07, 0.27, 0.26,  nan, 0.28, 0.29, 0.62, 0.46, 0.27],
        [0.  , 0.91, 0.  , 0.01,  nan, 0.  , 0.  , 0.01, 0.  , 0.  ],
        [0.  , 0.  , 0.72, 0.  ,  nan, 0.07, 0.  , 0.  , 0.  , 0.  ],
        [0.03, 0.02, 0.  , 0.73,  nan, 0.01, 0.  , 0.01, 0.02, 0.02],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.63, 0.01, 0.08, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.69, 0.01, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.01, 0.  , 0.27, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.01, 0.52, 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.7 ]],
       dtype=float32),
 array([[0.96, 0.07, 0.24, 0.26, 0.  , 0.3 , 0.06, 0.61, 0.09, 0.23],
        [0.  , 0.86, 0.  , 0.01, 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ],
        [0.  , 0.  , 0.65, 0.  , 0.  , 0.07, 0.  , 0.  , 0.  , 0.  ],
        [0.03, 0.02, 0.  , 0.73, 0.  , 0.01, 0.  , 0.01, 0.  , 0.0